In [265]:
%cd ~/advent_of_code_2024
from utils.get_input import get_input
data = get_input(15).strip()
warehouse,instructions = data.split("\n\n")
warehouse = warehouse.splitlines()
instructions = instructions.replace("\n","")

/home/arseny/advent_of_code_2024


In [266]:
# warehouse = ["########",
# "#..O.O.#",
# "##@.O..#",
# "#...O..#",
# "#.#.O..#",
# "#...O..#",
# "#......#",
# "########"]

# instructions = "<^^>>>vv<v>>v<<"

robot = (0,0)
boxes = set()
walls = set()
for i,row in enumerate(warehouse):
    for j,c in enumerate(row):
        if c=='O': boxes.add((i,j))
        elif c=='#': walls.add((i,j))
        elif c == '@': robot = (i,j)


def print_warehouse(walls,boxes,robot):
    for i,row in enumerate(warehouse):
        string = ""
        for j,c in enumerate(row):
            if (i,j) in walls:
                string += "#"
            elif (i,j) in boxes:
                string += "O"
            elif (i,j) == robot:
                string += "@"
            else:
                string += "."
        print(string)
        

In [267]:
directions = {">":(0,1),"^":(-1,0),"<":(0,-1),"v":(1,0)}
for direction in instructions:
    # print(direction)
    # print_warehouse(walls,boxes,robot)
    
    d = directions[direction]
    move_list = [robot]
    next_pos = (robot[0] + d[0], robot[1] + d[1])
    while next_pos in boxes:
        move_list.append(next_pos)
        next_pos = (next_pos[0] + d[0], next_pos[1] + d[1])
        
    if next_pos not in walls:
        robot = (move_list[0][0] + d[0], move_list[0][1] + d[1])
        for i in range(len(move_list)-1,0,-1):
            item = move_list[i]
            boxes.remove(item)
            boxes.add((item[0] + d[0], item[1] + d[1]))

total = 0
for box in boxes:
    total += (box[0]*100) + box[1]
total

1437174

In [268]:
robot = (0,0)
warehouse2 = {}
for i,row in enumerate(warehouse):
    for j,c in enumerate(row):
        if c=='O': 
            box = ((i,(j*2)),(i,(j*2)+1))
            warehouse2[(i,(j*2))] = box
            warehouse2[(i,(j*2)+1)] = box
        elif c=='#': 
            warehouse2[(i,(j*2))] = '#'
            warehouse2[(i,(j*2)+1)] = '#'
        elif c == '.': 
            warehouse2[(i,(j*2))] = '.'
            warehouse2[(i,(j*2)+1)] = '.'
        else:
            warehouse2[(i,(j*2))] = '@'
            warehouse2[(i,(j*2)+1)] = '.'
            robot = (i,(j*2))

def print_warehouse2(warehouse2):
    for i,row in enumerate(warehouse):
        string = ""
        for j,c in enumerate(row):
            c1, c2 = (i,j*2), (i,(j*2)+1)
            i1, i2 = warehouse2[c1], warehouse2[c2]
            if isinstance(i1,tuple):
                if i1[0] == c1:
                    i1, i2 = '[', ']'
                elif i1[1] == c1:
                    i1 = ']'
            if isinstance(i2,tuple):
                if i2[0] == c2:
                    i2 = '['
            string += i1 + i2
        print(string)

In [269]:
visited = set()
def move_box(box,d,warehouse2):
    move_list = [box]
    visited.add(box)
    (ibl,jbl),(ibr,jbr) = box
    
    new_r = warehouse2[(ibr + d[0], jbr + d[1])]
    if new_r not in visited:
        if new_r == "#": return False
        elif isinstance(new_r, tuple):
            result = move_box(new_r,d,warehouse2)
            if result: 
                move_list += result
            else: return False
                
    new_l = warehouse2[(ibl + d[0], jbl + d[1])]
    if new_l not in visited:
        if new_l == "#": return False
        elif isinstance(new_l, tuple):
            result = move_box(new_l,d,warehouse2)
            if result: 
                move_list += result
            else: return False

    return move_list

for direction in instructions:
    # print_warehouse2(warehouse2)
    # print(direction)
    
    visited, move_list = set(), []
    d = directions[direction]
    next_pos = (robot[0] + d[0], robot[1] + d[1])
    box_check = warehouse2[next_pos]

    if box_check == "#":
        continue
    elif isinstance(box_check, tuple):
        result = move_box(box_check, d, warehouse2)
        if result:
            move_list = result
        else: continue

    # creates sets for updating warehouse2
    old_points, new_points, new_points_l = set(), set(), set()
    for box in move_list:
        (ibl,jbl),(ibr,jbr) = box
        new_r = (ibr + d[0], jbr + d[1])
        new_l = (ibl + d[0], jbl + d[1])
        old_points.update([(ibl,jbl),(ibr,jbr)])
        new_points.update([new_l,new_r])
        new_points_l.add(new_l)

    # updates warehouse2
    empty_spaces = old_points - new_points
    for space in empty_spaces:
        warehouse2[space] = "."
    for point in new_points:
        if point in new_points_l:
            warehouse2[point] = (point,(point[0],point[1]+1))
        else:
            warehouse2[point] = ((point[0],point[1]-1),point)
        
    warehouse2[robot] = "."
    robot = next_pos
    warehouse2[robot] = "@"

total = 0
for i, row in enumerate(warehouse):
    for j in range((len(row)*2)-1):
        pos = warehouse2[(i,j)]
        if isinstance(pos,tuple):
            if pos[0] == (i,j):
                total += (i*100) + j
total
            

1437468